In [ ]:
import os
import sys

import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

import datacube
dc = datacube.Datacube()

In [ ]:
product = 's2_l2a_aws_cog'

In [ ]:
meas_info = dc.list_measurements()

In [ ]:
nodata_vals = meas_info.loc[product]['nodata']

In [ ]:
lat = (36.5, 37)
lon = (-76.5, -76)

In [ ]:
ds = dc.load(product=product, lat=lat, lon=lon,
               time=('2017-01-01', '2017-03-31'), 
               measurements=['red'], group_by='solar_day',
               dask_chunks=dict(time=1, latitude=2000, longitude=2000))

In [ ]:
ds

In [ ]:
ds = ds.compute()

In [ ]:
no_data_mask = xr.merge([ds[data_var] != nodata_vals[data_var] for data_var in ds]).to_array().all('variable')

In [ ]:
ds = ds.where(no_data_mask)

In [ ]:
min_ = np.nanpercentile(ds.red, 5)
max_ = np.nanpercentile(ds.red, 95)

ds = ds.where((min_<ds) & (ds<max_))

**Show the mean of the data**

In [ ]:
ds['red'].mean('time').plot.imshow()

**Show each day as an image**

In [ ]:
# for time in ds.time:
#     ds['red'].sel(time=time).plot.imshow()
#     plt.show()